In [1]:
from htm.bindings.sdr import SDR, Metrics
from htm.encoders.date import DateEncoder
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood
import numpy as np
import pandas as pd
import pathlib
import datetime
import csv
from datetime import datetime
import os
from htm.encoders.rdse import RDSE, RDSE_Parameters
import time
import traceback
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from tqdm import tqdm


In [2]:
class ReflexiveMemory:
  def __init__(self, reflexSize):
    self.acKey0 = None
    self.pairs = {}
    self.anomalyRM = []
    self.anomalyTM = []
    self.enableLearn = False
    self.tableSize  = reflexSize
    self.historyRM = []
    self.historyTM = []
    self.historyGT = []

  def add(self, denseColumns):
    acKey1 = '-'.join(map(str, denseColumns.sparse))
    if(self.acKey0 != None):

      sequence = self.pairs.get(self.acKey0, {})
      sequence_data = sequence.get(acKey1, {
         "count": 0,
         "time": datetime.now()
      })
      sequence_data["count"] = sequence_data["count"] + 1
      sequence_data["time"] = datetime.now()

      if self.pairs.get(self.acKey0, None) is None:
        self.pairs[self.acKey0] = { acKey1: sequence_data }
      else:
        self.pairs[self.acKey0][acKey1] = sequence_data

      table_entries = 0
      oldKey1 = None
      oldKey2 = None
      oldTime = datetime.now()
      for key1, value1 in self.pairs.items():
        table_entries = table_entries + len(value1.items())
        for key2, value2 in value1.items():
          if value2['time'] < oldTime:
            oldKey1 = key1
            oldKey2 = key2
            oldTime = datetime.now()
      if table_entries > self.tableSize:
        self.enableLearn = True
        del self.pairs[oldKey1][oldKey2]
        if len(self.pairs[oldKey1].items()) == 0:
          del self.pairs[oldKey1]

    self.acKey0 = acKey1

  def predict(self, denseColumns):
    return_count = 0
    return_sdr = None

    acKey = '-'.join(map(str, denseColumns.sparse))
    sequences = self.pairs.get(acKey, {})
    for sequence_key, sequence_data in sequences.items():
      if sequence_data["count"] > return_count:
        return_count = sequence_data["count"]
        return_sdr = sequence_key

    if return_sdr is not None:
      tmp_sdr = SDR( denseColumns.dimensions )
      tmp_sdr.sparse = list(map(int, return_sdr.split('-')))
      return_sdr = tmp_sdr
    else:
      return_count = None

    return return_count, return_sdr

  def learn(self, denseColumns1, sp, tm):

    tm.activateDendrites(True)
    predictiveColumns = SDR( tm.getPredictiveCells().dimensions[0] )
    predictiveColumns.sparse = list(set(sorted(list(np.where(tm.getPredictiveCells().dense == 1)[0]))))

    activeColumns1 = SDR( predictiveColumns.dimensions )
    sp.compute(denseColumns1, False, activeColumns1)

    denseColumns0 = SDR( denseColumns1.dimensions )
    denseColumns0.sparse = list(map(int, self.acKey0.split('-')))
    activeColumns0 = SDR( predictiveColumns.dimensions )
    sp.compute(denseColumns0, False, activeColumns0)
 
    reflexiveCount, denseReflexiveColumns = self.predict(denseColumns0)
    reflexiveColumns = SDR( predictiveColumns.dimensions )
    if denseReflexiveColumns is not None:
      sp.compute(denseReflexiveColumns, False, reflexiveColumns)    

    overlapRM = 1 - np.count_nonzero((reflexiveColumns.dense & activeColumns1.dense)) / np.count_nonzero(activeColumns1.dense)
    self.anomalyRM.append( overlapRM )

    overlapTM = 1 - np.count_nonzero((predictiveColumns.dense & activeColumns1.dense)) / np.count_nonzero(activeColumns1.dense)
    self.anomalyTM.append( overlapTM )

    self.historyRM.append( reflexiveColumns.dense )
    self.historyTM.append( predictiveColumns.dense )
    self.historyGT.append( activeColumns1.dense )

  def compute(self, denseColumns, sp, tm):
    if self.acKey0 is not None:
      self.learn(denseColumns, sp, tm)  
    self.add(denseColumns)
  

In [3]:
inputSources = [
   "monthly_sp500.csv",
   "weekly_dow_jones.csv",
   "weekly_nasdaq.csv",
   "weekly_sp500.csv",
   "monthly_vix_close.csv",
   "monthly_vix_high.csv",
   "monthly_vix_low.csv",
   "monthly_vix_open.csv",
   "daily_natural_gas.csv",
   "daily_oil_prices.csv",
   "value1_vix_close.csv",
   "value1_vix_high.csv",
   "value1_vix_low.csv",
   "value1_vix_open.csv"
]

In [4]:

config = {
    'enc': {
        "value" :
            {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
        "time": 
            {'timeOfDay': (30, 1), 'weekend': 21}
    },
    'sp': {
        'inputDimensions': None,
        'columnDimensions': (1638,),
        'potentialPct': 0.85,
        'potentialRadius': None,
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False,
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    },
    'anomaly': {'period': 1000},
    'learnRows': 100,
    'reflexSize': 128
}


In [5]:
def anomalyScore(x, y):
    if np.count_nonzero(y) != 0:
        return 1 - np.count_nonzero((x & y)) / np.count_nonzero(y)
    return 1

def match(y, x, idx1):
  overlap_min = 0.2
  n_samples = len(y)
  score1 = anomalyScore(y[idx1], x[idx1])
  if score1 > (1 - overlap_min):
     idx_closest = None
     score_closest = None
     for idx2 in range(n_samples):
        score2 = anomalyScore(y[idx2], x[idx1])
        if score_closest is None or score_closest > score2:
            score_closest = score2
            idx_closest = idx2
     return idx_closest
  return idx1

In [6]:
input_path = pathlib.Path('../datasets/numenta')
dataset_metrics = []

pbar = tqdm(total=len(inputSources))
for dataset in inputSources:

    df_loop_start = time.time()

    records = []
    with open(input_path.joinpath(dataset), "r") as fin:
        reader = csv.reader(fin)
        headers = next(reader)
        next(reader)
        next(reader)
        for record in reader:
            records.append(record)
        
    scalarEncoderParams = RDSE_Parameters()
    scalarEncoderParams.size = config["enc"]["value"]["size"]
    scalarEncoderParams.sparsity = config["enc"]["value"]["sparsity"]
    scalarEncoderParams.resolution = config["enc"]["value"]["resolution"]
    scalarEncoder = RDSE( scalarEncoderParams )
    encodingWidth = (scalarEncoder.size)

    config['sp']['inputDimensions'] = (encodingWidth,)
    config['sp']['potentialRadius'] = encodingWidth

    sp = SpatialPooler(
        inputDimensions = config['sp']['inputDimensions'],
        columnDimensions = config['sp']['columnDimensions'],
        potentialPct = config['sp']['potentialPct'],
        potentialRadius = config['sp']['potentialRadius'],
        globalInhibition = config['sp']['globalInhibition'],
        localAreaDensity = config['sp']['localAreaDensity'],
        synPermInactiveDec = config['sp']['synPermInactiveDec'],
        synPermActiveInc = config['sp']['synPermActiveInc'],
        synPermConnected = config['sp']['synPermConnected'],
        boostStrength = config['sp']['boostStrength'],
        wrapAround = config['sp']['wrapAround'],
        seed = config['sp']['seed']
    )

    tm = TemporalMemory(
        columnDimensions = config['sp']['columnDimensions'],
        cellsPerColumn = config['tm']['cellsPerColumn'],
        activationThreshold = config['tm']['activationThreshold'],
        initialPermanence = config['tm']['initialPermanence'],
        connectedPermanence = config['sp']['synPermConnected'],
        minThreshold = config['tm']['minThreshold'],
        maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
        permanenceIncrement = config['tm']['permanenceIncrement'],
        permanenceDecrement = config['tm']['permanenceDecrement'],
        predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
        maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
        maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
    )

    rm = ReflexiveMemory( config['reflexSize'] )

    enc_info = Metrics( [encodingWidth], 999999999)
    sp_info = Metrics( sp.getColumnDimensions(), 999999999 )
    tm_info = Metrics( [tm.numberOfCells()], 999999999 )
    anomaly_history = AnomalyLikelihood(config["anomaly"]["period"])

    inputs = []
    anomaly = []
    anomalyProb = []
   
    try:
        
        for count, record in enumerate(records):

            consumption = float(record[1])

            inputs.append( consumption )
            consumptionBits = scalarEncoder.encode(consumption)

            encoding = SDR( consumptionBits )
            enc_info.addData( encoding )
            
            activeColumns = SDR( sp.getColumnDimensions() )

            learn_sp = config['sp']['learn']
            learn_tm = config['tm']['learn']
            if count < config['learnRows']:
                learn_sp = True
                learn_tm = True

            rm.compute(encoding, sp, tm)

            sp.compute(encoding, learn_sp, activeColumns)
            sp_info.addData( activeColumns )

            tm.compute(activeColumns, learn=learn_tm)
            tm_info.addData( tm.getActiveCells().flatten() )

            anomaly.append( tm.anomaly )
            anomalyProb.append( anomaly_history.compute(tm.anomaly) )

    except Exception as e:
        print(traceback.format_exc())
        print(e)

    metric = {}
    metric['dataset'] = dataset
    metric['dataset-time'] = time.time() - df_loop_start
    
    n_samples = len(rm.historyGT)
    Y_labels = list(range(n_samples))

    X_labels = [ match(rm.historyGT, rm.historyRM, idx) for idx in range(n_samples)]
    precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
    metric['precision-rm'] = precision
    metric['recall-rm'] = recall
    metric['fscore-rm'] = fscore
    metric['support-rm'] = support
    metric['accuracy-rm'] = accuracy_score(Y_labels, X_labels)
    metric['anomaly-avg-rm'] = sum(rm.anomalyRM) / len(rm.anomalyRM)
    metric['anomaly-samples-rm'] = len(rm.anomalyRM)

    X_labels = [ match(rm.historyGT, rm.historyTM, idx) for idx in range(n_samples)]
    precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
    metric['precision-tm'] = precision
    metric['recall-tm'] = recall
    metric['fscore-tm'] = fscore
    metric['support-tm'] = support
    metric['accuracy-tm'] = accuracy_score(Y_labels, X_labels)
    metric['anomaly-avg-tm'] = sum(rm.anomalyTM) / len(rm.anomalyTM)
    metric['anomaly-samples-tm'] = len(rm.anomalyTM)

    dataset_metrics.append(metric)
    pbar.update(1)

pbar.close()

100%|██████████| 14/14 [01:44<00:00,  7.46s/it]


In [7]:
df = pd.DataFrame(dataset_metrics)
df.to_csv('metrics.csv', index=False)
df

,dataset,dataset-time,precision-rm,recall-rm,fscore-rm,support-rm,accuracy-rm,anomaly-avg-rm,anomaly-samples-rm,precision-tm,recall-tm,fscore-tm,support-tm,accuracy-tm,anomaly-avg-tm,anomaly-samples-tm
0,monthly_sp500.csv,1.229278,0.557225,0.558074,0.557320,None,0.558074,0.513244,1765,0.778379,0.791501,0.782442,None,0.791501,0.277849,1765
1,weekly_dow_jones.csv,2.086443,0.008738,0.010101,0.008979,None,0.010101,0.994375,2079,0.047327,0.060125,0.050853,None,0.060125,0.936301,2079
2,weekly_nasdaq.csv,2.174645,0.038141,0.039423,0.038382,None,0.039423,0.976863,2080,0.298511,0.329808,0.307470,None,0.329808,0.758574,2080
3,weekly_sp500.csv,1.979575,0.070399,0.071600,0.070640,None,0.071600,0.955577,2081,0.424369,0.463719,0.436007,None,0.463719,0.623372,2081
4,monthly_vix_close.csv,2.877190,0.812268,0.814815,0.812583,None,0.814815,0.401766,4050,0.955679,0.961235,0.957009,None,0.961235,0.058193,4050
5,monthly_vix_high.csv,3.455111,0.796688,0.799506,0.796997,None,0.799506,0.422154,4050,0.859226,0.866420,0.860600,None,0.866420,0.074667,4050
6,monthly_vix_low.csv,2.745371,0.857953,0.860000,0.858315,None,0.860000,0.336735,4050,0.878981,0.883210,0.879737,None,0.883210,0.035840,4050
7,monthly_vix_open.csv,3.223168,0.780952,0.783704,0.781308,None,0.783704,0.428433,4050,0.875527,0.882469,0.877098,None,0.882469,0.066163,4050
8,daily_natural_gas.csv,1.980971,0.996905,0.997068,0.996914,None,0.997068,0.061880,5798,0.998099,0.998448,0.998175,None,0.998448,0.008085,5798
9,daily_oil_prices.csv,3.426489,0.506325,0.506506,0.506345,None,0.506506,0.565395,8300,0.994738,0.996024,0.995127,None,0.996024,0.019858,8300
